# Introduction

**TODO**

# Data Extraction

In [ ]:
# Install Packages
# You might need to install r before running this script

using Pkg

Pkg.add("CSV")
Pkg.add("DataFrames")
Pkg.add("DataFramesMeta")
Pkg.add("StringCases")

In [19]:
using CSV
using DataFrames
using DataFramesMeta
using StringCases

In [ ]:
trips = CSV.read("202205-divvy-tripdata.csv", DataFrame)
stations = CSV.read("Divvy_Bicycle_Stations.csv", DataFrame)

# rename upper case sepearated by space to lower case separated by underscore

function snake_case(camelstring::S) where S<:AbstractString
    wordpat = r"
    ^[a-z]+ |                  #match initial lower case part
    [A-Z][a-z]+ |              #match Words Like This
    \d*([A-Z](?=[A-Z]|$))+ |   #match ABBREV 30MW 
    \d+                        #match 1234 (numbers without units)
    "x
  
    smartlower(word) = any(islowercase, word) ? lowercase(word) : word
    words = [smartlower(m.match) for m in eachmatch(wordpat, camelstring)]
  
    join(words, "_")
end

for name in names(stations)
    rename!(stations, name => snake_case(name))
end

rename!(stations, "ID" => "id")
@transform!(stations, :id = string.(stations.id))

In [70]:
using DataFramesMeta

station_start = @chain trips begin
    groupby(:start_station_id)
    @combine(n = length(:ride_id))
end

station_end = @chain trips begin
    groupby(:end_station_id)
    @combine(n = length(:ride_id))
end

@rename!(station_start, :id = :start_station_id)
@rename!(station_start, :n_start = :n)
@rename!(station_end, :id = :end_station_id)
@rename!(station_end, :n_end = :n)

dropmissing!(station_start)
dropmissing!(station_end)

station = @chain station_start begin
    outerjoin(station_end, on = :id)
    innerjoin(stations, on = :id)
    @select(:id, :n_start, :n_end, :latitude, :longitude)
end

replace!(station.n_start, missing => 0)
replace!(station.n_end, missing => 0)

station

┌ Warning: Using an un-quoted Symbol on the LHS is deprecated. Write :n = ... instead.
└ @ DataFramesMeta C:\Users\hongtao\.julia\packages\DataFramesMeta\nI3Y1\src\parsing.jl:373
┌ Warning: Using an un-quoted Symbol on the LHS is deprecated. Write :n = ... instead.
└ @ DataFramesMeta C:\Users\hongtao\.julia\packages\DataFramesMeta\nI3Y1\src\parsing.jl:373


Row,id,n_start,n_end,latitude,longitude
,String,Int64?,Int64?,Float64,Float64
1,641,2,2,41.9142,-87.7168
2,367,2,3,41.8307,-87.6562
3,648,1,1,41.7799,-87.6509
4,427,5,1,41.7805,-87.606
5,644,580,598,41.8686,-87.6862
6,503,9,10,41.9248,-87.7145
7,643,251,283,41.892,-87.6894
8,640,1,1,41.9499,-87.7139
9,658,544,438,41.903,-87.6838


In [71]:
@chain station begin
    @combine(supply = sum(:n_start), demand = sum(:n_end))
end

┌ Warning: Using an un-quoted Symbol on the LHS is deprecated. Write :supply = ... instead.
└ @ DataFramesMeta C:\Users\hongtao\.julia\packages\DataFramesMeta\nI3Y1\src\parsing.jl:373
┌ Warning: Using an un-quoted Symbol on the LHS is deprecated. Write :demand = ... instead.
└ @ DataFramesMeta C:\Users\hongtao\.julia\packages\DataFramesMeta\nI3Y1\src\parsing.jl:373


Row,supply,demand
,Int64,Int64
1,28707,27371


# Locations of Divvy Stations



<p align="center">
  <img src="stations_divvy.svg" alt="Divvy Stations"/>
</p>